In [3]:
!pip install gradio
!pip install opencv-python
!pip install matplotlib
!pip install sklearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing in

In [5]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image, ImageDraw, ImageFont
import gradio as gr

# Function to classify the undertone of the dominant color
def classify_undertone(rgb_color):
    r, g, b = rgb_color
    if r > g and g > b:
        return "Warm"
    elif b > r and b > g:
        return "Cool"
    else:
        return "Neutral"

# Function to find the dominant skin tone from the image
def find_dominant_skin_tone(image, n_clusters=3):
    # Convert image from PIL format to a numpy array in RGB
    image_array = np.array(image)

    # Reshape the image to a 2D array of pixels
    pixel_data = image_array.reshape((-1, 3))

    # Apply k-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(pixel_data)

    # Get the cluster centers (dominant colors)
    dominant_colors = kmeans.cluster_centers_.astype(int)

    # Identify the dominant skin tone cluster
    labels = kmeans.labels_
    skin_cluster = np.argmax(np.bincount(labels))

    # Get the dominant skin tone color
    skin_tone_color = dominant_colors[skin_cluster]

    # Classify the skin undertone
    undertone = classify_undertone(skin_tone_color)

    # Define color palettes and names for suggestions
    cool_palette = [
        (255, 192, 203),  # Pastel Pink
        (240, 200, 220),  # Blushing Pink
        (255, 105, 180),  # Fuchsia
        (240, 128, 128),  # Soft Coral
        (152, 217, 224),  # Minty Fresh
        (188, 212, 196),  # Soft Sage
        (204, 255, 204),  # Pale Lime
        (144, 238, 144),  # Icy Green
        (230, 190, 240),  # Lavender Dream
        (221, 204, 236),  # Lilac Mist
        (204, 153, 204),  # Pastel Purple
        (255, 0, 128),    # Raspberry Red
        (255, 105, 180),  # Cherry Red
        (220, 20, 60),    # Cranberry Red
        (240, 128, 128)   # Blushing Red
    ]
    cool_palette_names = [
        "Pastel Pink", "Blushing Pink", "Fuchsia", "Soft Coral",
        "Minty Fresh", "Soft Sage", "Pale Lime", "Icy Green",
        "Lavender Dream", "Lilac Mist", "Pastel Purple",
        "Raspberry Red", "Cherry Red", "Cranberry Red", "Blushing Red"
    ]

    warm_palette = [
        (255, 218, 185),  # Peachy Keen
        (240, 215, 180),  # Golden Honey
        (255, 127, 80),   # Soft Coral
        (245, 245, 220),  # Warm Beige
        (255, 192, 128),  # Sunset Glow
        (230, 190, 150),  # Caramel Delight
        (255, 255, 240),  # Ivory Dream
        (255, 204, 153),  # Peach Sorbet
        (240, 230, 200),  # Golden Sand
        (255, 229, 204),  # Soft Peach
        (245, 245, 235),  # Warm Cream
        (230, 180, 120),  # Turmeric Spice
        (255, 192, 80),   # Cantaloupe
        (240, 220, 180),  # Honeycomb
        (255, 224, 189)   # Peachy Blush
    ]
    warm_palette_names = [
        "Peachy Keen", "Golden Honey", "Soft Coral", "Warm Beige",
        "Sunset Glow", "Caramel Delight", "Ivory Dream", "Peach Sorbet",
        "Golden Sand", "Soft Peach", "Warm Cream", "Turmeric Spice",
        "Cantaloupe", "Honeycomb", "Peachy Blush"
    ]
    neutral_palette = [
        #Caramel
        (245, 222, 179),
        #Coffee
        (139, 69, 19),
        #Taupe
        (135, 121, 105),
        #Soft Pink
        (255, 192, 203),
        #Mauve
        (224, 176, 255),
        #Dusty Blush
        (245, 183, 177),
        #Champagne
        (247, 231, 206),
        #Mint
        (153, 255, 204),
        #Ivory
        (255, 255, 240),
        #Blue Lagoon
        (0, 191, 255),
        #Navy
        (0, 0, 128),
        #Dark Purple
        (75, 0, 130),
        #Lavender
        (230, 190, 255),
        #Red
        (255, 0, 0),
        #Burgundy
        (139, 0, 0)

    ]
    neutral_palette_names = [
        "Caramel", "Coffee", "Taupe", "Soft Pink", "Mauve", "Dusty Blush",
        "Champagne", "Mint", "Ivory", "Blue Lagoon", "Navy", "Dark Purple",
        "Red", "Burgundy"
    ]

    # Select palette based on undertone
    if undertone == "Warm":
        selected_palette = warm_palette
        palette_names = warm_palette_names
    elif undertone == "Cool":
        selected_palette = cool_palette
        palette_names = cool_palette_names
    else:
        selected_palette = neutral_palette
        palette_names = neutral_palette_names

    # Create a palette image with increased width and height
    palette_height = 300  # Increased height to accommodate vertical text
    palette_width = 500   # Increased width
    color_block_width = palette_width // len(selected_palette)  # Adjusted block width
    palette_image = np.zeros((palette_height, palette_width, 3), dtype=np.uint8)

    # Draw color blocks and vertical names
    for i, (color, name) in enumerate(zip(selected_palette, palette_names)):
        palette_image[:, i*color_block_width:(i+1)*color_block_width] = color
        # Convert to PIL image for drawing text
        pil_image = Image.fromarray(palette_image)
        draw = ImageDraw.Draw(pil_image)
        font = ImageFont.load_default()  # Load default font
        text = name
        # Draw text on the palette image vertically
        text_width, text_height = draw.textsize(text, font=font)
        # Draw vertically
        for j, char in enumerate(text):
            draw.text((i*color_block_width + 5, j*text_height), char, fill=(0, 0, 0), font=font)
        palette_image = np.array(pil_image)

    # Convert the palette to an image
    palette_image = Image.fromarray(palette_image)

    # Return the RGB color of the skin tone and the selected palette image
    return palette_image, f"Dominant Skin Tone RGB: {tuple(skin_tone_color)}, Undertone: {undertone}", suggest_dresses(undertone)

# Function to suggest dresses based on undertone
def suggest_dresses(undertone):
    suggestions = []

    # Dress suggestions based on skin tone undertone
    if undertone == "Warm":
        suggestions = [
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/24771432/2023/10/20/d3ca659a-3555-45d3-a540-5a5f2fe587951697775523692-Antheaa-Yellow-Checked-Chiffon-Fit--Flare-Dress-817169777552-7.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/24959496/2023/10/16/c3069ccc-028a-414f-904c-34816f7f35c41697456929953-Stylecast-X-Hersheinbox-Floral-Print-Chiffon-Maxi-Dress-2316-1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/17171788/2022/3/2/79806943-5b90-485c-920b-1b6371e682b21646218466371-QUIERO-Women-Dresses-1441646218465761-1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/16042862/2024/1/27/4833d882-0ae2-4fc6-a635-5e8d78634cb71706332837025-20Dresses-Beige-Floral-Chiffon-Empire-Dress-2161706332836669-1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/27373914/2024/2/6/06ebd462-b580-429d-8609-c0208bf72b3a1707213040717AntheaaFloralPrintPuffSleeveChiffonFitFlareMidiDress1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/19442062/2022/8/9/df789b5d-3cee-43b2-a97f-944c248405951660019752856GUFRINACamelBrownFloralGeorgetteA-LineDress1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/19628870/2022/8/24/520cf7bb-1578-484c-944a-1ec9d2f4d1991661282657161Dresses1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/19471206/2022/8/11/0b111fb5-7e46-4069-8aa6-e4c50ad367d31660212842184Dresses1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/20980410/2022/11/29/97ec5a16-b2b5-4e88-9c97-0931b98798fe1669715431475OliviaCottonMaxiDress1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/17171788/2022/3/2/79806943-5b90-485c-920b-1b6371e682b21646218466371-QUIERO-Women-Dresses-1441646218465761-1.jpg"}

        ]
    elif undertone == "Cool":
        suggestions = [
             {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/21554756/2023/1/25/3a7e68cd-ad5a-4c8c-b32e-f7801ab4c0041674631722939-Deewa-Women-Dresses-7421674631722127-1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/25448734/2023/10/18/696e8cb2-3d15-4c9b-8570-60a562c35d701697617879885-DressBerry-Women-Dresses-5011697617879529-1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/24968446/2023/9/14/e86b1567-344d-4a66-99df-870ed0c48eb91694698367699StrongAndBraveLavenderStripedPuffSleeveGeorgetteFitFlareMidi1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/16831952/2023/9/20/6fddc3b7-1654-427a-82bb-0717b11a22cb1695211659339TokyoTalkiesWomenPurpleTieredStrappyDress1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/20552936/2022/10/31/212cc0af-1157-4c35-81a5-5bb0e3a377e01667198131711ADDYVEROFoldPleatedColdShoulderBodyconDress1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/15754220/2022/5/20/c7c15876-7817-4463-ab4c-54d2f79255791653042283621-Magenta-Dress-With-Embroidery-9051653042283014-1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/26896892/2024/1/12/43596166-5873-46f9-9925-e5bfb2249fc81705070070946AntheaaRedChiffonFitFlareDress1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/24333012/2023/8/3/06af99c5-e0dd-4598-8d04-32e6e05f81851691083570650plusSRedFloralPrintFitFlareDress1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/26001348/2023/11/23/a21dc989-4993-41ba-b724-7f8956122b781700682371480AllenSollyWomanPinkFormalDress1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/h_1440,q_100,w_1080/v1/assets/images/13719160/2021/3/5/b04a5dc0-9beb-4490-ae2c-f0f9654ded1c1614925067822-Athena-Fuchsia-Pink-dress-with-Power-shoulderS--Tulip-Bottom-1.jpg"}
        ]
    else:
        suggestions = [
            {"description": "", "image_url": "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/2024/JULY/25/tP5RIm01_93a97f9d6b70465f925102e825808b32.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/21428808/2023/1/5/7f2b24e2-e7fb-4781-b887-e46eda15c4371672935150655AASKCoffeeBrownStripedCrepeShirtMidiDress1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/11866156/2023/5/30/3edbc5be-45a2-4e5e-8d77-5df41ac87c781685429990619-SASSAFRAS-Brown--Black-Paisley-Print-Tiered-Maxi-Dress-15516-1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/19003158/2022/7/7/1137b774-7137-450d-be86-d0c77d237e5d1657187230521AASKWomensElegantBeltedPleatedSleeveessShortDressPinkColor1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/17412678/2022/3/16/2cbcd8a9-18f8-4143-a34b-283789ec037d1647426553276-Varanga-Women-Mauve-Solid-Anarkali-Kurta-8601647426552783-1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/15621300/2021/9/25/a7a235ac-e7b4-4ca6-b118-3441b05dd4181632584129982StyleStoneBlueChiffonA-LineDress1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/19438944/2022/8/8/a558243a-e358-4cf5-b624-58f7956764d31659974725083Tops1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/19296880/2022/8/17/8c197536-f87d-41f2-a3b0-11db3ed2c8cb1660730304770CottinfabBurgundyVNeckMaxiWrapDress1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/23359124/2023/5/23/9206b519-89ed-4990-b657-d368d6f77cb61684866245279MINTSTREETMulticolouredFloralPrintCrepeFitFlareMidiDress1.jpg"},
            {"description": "", "image_url": "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/14101562/2021/6/11/a8a45c73-594f-4362-a318-3447a77a429f1623387732886CLNAVY1103SKurtaSetsAHIKAWomenJeansRoadsterWomenJeansRoadste1.jpg"},
        ]

    return suggestions

# Gradio Interface function
def analyze_and_suggest(image):
    # Convert the image to RGB
    image = np.array(image)

    # Analyze dominant skin tone
    palette_image, undertone_info, dress_suggestions = find_dominant_skin_tone(image)

    # Prepare suggestions output
    # Remove descriptions from the output
    images_output = [s['image_url'] for s in dress_suggestions]

    return palette_image, undertone_info, images_output

# Gradio Interface
iface = gr.Interface(
    fn=analyze_and_suggest,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Image(type="pil", label="Selected Color Palette"),
        gr.Textbox(label="Dominant Skin Tone and Undertone"),
        gr.Gallery(label="Dress Images")  # Updated to remove the dress description textbox
    ],
    title="Skin Tone and Dress Recommendations",
    description="Upload an image to find the dominant skin tone, its name, undertone, and receive dress recommendations based on the color palette."
)

# Launch the interface
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://07a16c3ae56d8c8a54.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
